In [6]:
import pandas as pd
from transformers import pipeline
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
classifier = pipeline(
    'text-classification',
    model='./distilbert-stance-model',
    device=0 if torch.cuda.is_available() else -1,
    batch_size=256,
    truncation=True,
    max_length=384
)

Device set to use cuda:0


In [3]:
chunk_size = 256000
all_results = []

for chunk in tqdm(pd.read_csv('data/youtube_comments.csv', chunksize=chunk_size)):
    predictions = classifier(chunk['comment'].tolist())
    chunk['stance_label'] = [p['label'] for p in predictions]
    all_results.append(chunk)

7it [1:50:57, 951.04s/it] 


In [4]:
df_final = pd.concat(all_results, ignore_index=True)
df_final.to_csv('full_dataset_with_stance_distilbert.csv', index=False)